In [1]:
!pip install facenet_pytorch
!pip install matplotlib

import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from PIL import Image
import cv2
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

import warnings
warnings.filterwarnings("ignore")

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)


model.to(DEVICE)
model.eval()



criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy for single-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


#l1_strength = 0.001  # Adjust this parameter based on your preference
# Optimizer with L1 Regularization
#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=l1_strength)




transform = transforms.Compose([
    transforms.Resize(299),  # InceptionResNetV1 input size
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
mtcnn = MTCNN(
    select_largest=False,
    post_process=False,
    device=DEVICE
).to(DEVICE).eval()



class CustomImageDataset(Dataset):
    def __init__(self,  img_dir, transform=None, target_transform=None):
        #self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        real = img_dir+"Real"
        fake = img_dir + "Fake"
        self.images = []
        self.labels = []
        
        for image_file in os.listdir(real):
            image = Image.open(os.path.join(real, image_file))
           
            self.images.append(image)
            self.labels.append(1)
          
        for image_file in os.listdir(fake):
            image = Image.open(os.path.join(fake, image_file))
           
            self.images.append(image)
            self.labels.append(0)

        self.img_labels = self.labels
    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform is not None:
            image = self.transform(image)

        return image, label


train_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Train/"

training_data = CustomImageDataset( train_dir , transform)


print(len(training_data))

from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)


num_epochs = 30



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.3 MB/s eta 0:00:00


  0%|          | 0.00/107M [00:00<?, ?B/s]

39428


In [2]:
for epoch in range(num_epochs):
    torch.save(model, 'epoch' + str(epoch) + 'deepfake_detection.pt')
    model.train()
    running_loss = 0.0
    


    for inputs, labels in train_dataloader:
        
      inputs = torch.tensor(inputs)
      labels = torch.tensor(labels)
      inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs.view(-1), labels.float())
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')




Epoch [1/30], Loss: 0.1661
Epoch [2/30], Loss: 0.0944
Epoch [3/30], Loss: 0.0608
Epoch [4/30], Loss: 0.0480
Epoch [5/30], Loss: 0.0411
Epoch [6/30], Loss: 0.0486
Epoch [7/30], Loss: 0.0306
Epoch [8/30], Loss: 0.0367
Epoch [9/30], Loss: 0.0299
Epoch [10/30], Loss: 0.0342
Epoch [11/30], Loss: 0.0155
Epoch [12/30], Loss: 0.0290
Epoch [13/30], Loss: 0.0294
Epoch [14/30], Loss: 0.0145
Epoch [15/30], Loss: 0.0132
Epoch [16/30], Loss: 0.0136
Epoch [17/30], Loss: 0.0121
Epoch [18/30], Loss: 0.0110
Epoch [19/30], Loss: 0.0114
Epoch [20/30], Loss: 0.0111
Epoch [21/30], Loss: 0.0082
Epoch [22/30], Loss: 0.0089
Epoch [23/30], Loss: 0.0194
Epoch [24/30], Loss: 0.0192
Epoch [25/30], Loss: 0.0063
Epoch [26/30], Loss: 0.0181
Epoch [27/30], Loss: 0.0241
Epoch [28/30], Loss: 0.0060
Epoch [29/30], Loss: 0.0044
Epoch [30/30], Loss: 0.0057
